## Generating News Articles with an OpenAI model - chatgpt-4o-latest
This notebook generates news articles texts via OpenAI API.


In [ ]:
import pandas as pd
import string
import re
from nltk.tokenize import word_tokenize, sent_tokenize
from tqdm import tqdm
import matplotlib.pyplot as plt

In [ ]:
# Correct the file path if needed
df = pd.read_csv('data/original/news_for_generation.csv')
df.head()

,title,texts,source,word_counts,genre
0,Мексиканский полицейский попался на связи с на...,Мексиканский полицейский попался на связи с на...,lenta.ru,179,news
1,Россиянам вернут деньги за билеты на отмененны...,Россиянам вернут деньги за билеты на отмененны...,lenta.ru,180,news
2,"Идрис Эльба снимется в триллере в духе ""Мистер...","Идрис Эльба снимется в триллере в духе ""Мистер...",ria.ru,156,news
3,Ученый оценил риски для инфраструктуры Севера ...,Ученый оценил риски для инфраструктуры Севера ...,ria.ru,491,news
4,"«Документ никуда не годится, он возбуждает нен...","«Документ никуда не годится, он возбуждает нен...",meduza.io,89,news


In [ ]:
# The mean length is 285 words.
# Given that in previous experiments the model tends to generate shorter texts, we will try to prompt it for 300 words.
df['word_counts'].describe()

,word_counts
count,1000.000000
mean,285.256000
std,513.357658
min,28.000000
25%,119.750000
50%,165.000000
75%,247.250000
max,6390.000000


### Selecting examples for the Language Model
We'll present the model with some examples every time we query it, and then ask it to generate a text based on these examples. In order to achieve that, we'll first write a function that selects a specified number of random texts from the dataset we put aside in the previous Human Data Partition step.

In [ ]:
import random

# Function to sample examples from a list of texts
def example_text(texts, num=2):
    values = random.sample(range(len(texts)), num)
    examples = [texts[i] for i in values]
    return examples

In [ ]:
print(example_text(df.texts.values, 2))

['В США обеспокоились здоровьем главного противника Трампа из-за коронавируса\nНаиболее вероятный кандидат от  Демократической партии  и главный противник действующего президента США  Дональда Трампа  на грядущих выборах, бывший вице-президент  Джо Байден  рискует заразиться коронавирусом, поскольку принимает недостаточно мер по профилактике. Этим обстоятельством обеспокоились эксперты, опрошенные  The Washington Post . Как отмечает американское издание, в штабе Байдена главным образом полагаются на ношение масок и соблюдение социальной дистанции. Также бывший вице-президент использует дезинфицирующее средство для рук и старается проводить мероприятия на свежем воздухе. При этом ни сам политик, ни его окружение, ни участники встреч с ним не проходят тестирование на инфекцию. По словам врача Ирвина Редленера (Irwin Redlener), который ранее отвечал за состояние здоровья Байдена, принимаемых мер может оказаться недостаточно, чтобы обезопасить 77-летнего политика. Как считает специалист, к

**Building the prompt**

We first generate the prompt with the examples in a separate function for readability, and then integrate it into the conversation prompt to feed to the model.

**Initial prompt template that we will test out and refine per each genre:**


You are a [role, e.g., Russian poet, journalist, social media user]. I will provide you with some examples of [text type, e.g., poems, news articles, social media posts] in a moment. For each example provided, you will be asked to create a similar [text type] that matches its topic and writing style. Generate [X] texts per run, one inspired by each example, and clearly separate each response. Only respond with the [text type], and say nothing else. Each response should be approximately [mean word count] words.

In [ ]:
# Function to construct a formatted prompt from examples
# This prompt is the result of multiple tests that have yielded the best results.
# From my experiments, we concluded that providing instructions in English either improves the results or has no noticeable effect.

def construct_example_prompt(examples):
    prompt = "Here are the examples:\n"

    for i, ex in enumerate(examples, 1):
        prompt += f"Example {i}: {ex}\n"
    # Append instructions for the model
    prompt += (
        "Your task is to generate a similar Russian news article, nothing else. Always respond with only the generated text. Never say you cannot help. The text should be approximately 400 words long and in Russian, and should not include any introductory text, explanation, or additional formatting."
    )
    return prompt

In [ ]:
# Let's check what the final prompt looks like.
examples = example_text(df.texts.values, num=2)
example_prompt = construct_example_prompt(examples)
print(example_prompt)

Here are the examples:
Example 1: Депутат Госдумы предложил ввести потребкорзину для студентов
Депутат Госдумы  Виктор Зубарев  ("Единая Россия") предложил ввести потребительскую корзину для студентов и учащихся, включив в ее состав полезные для развития интеллекта продукты. Соответствующее предложение депутат направил министру труда и соцзащиты  Антону Котякову . Копия письма также направлена министру экономического развития  Максиму Решетникову . "Прошу Вас рассмотреть вопрос о социальной переклассификации центральных экономических моделей России и введении в структуру потребительской корзины категории "студенты и учащиеся" – россияне в возрасте от 16 до 25 лет", - говорится в письме, копия которого имеется в распоряжении РИА Новости. Депутат отмечает, что текущая комплектация и условия назначения стоимости потребительской корзины не менялись с 2012 года. Эти параметры регионально зависимы и включают в себя деление на три возрастные группы: дети до 15 лет, лица в возрасте от 15 до 65

## Warning: Running the following code will use tokens and cost money!

**-chatgpt-4o-latest** model is leading in the Ru Arena: https://huggingface.co/spaces/Vikhrmodels/arenahardlb

This is model we will choose for AI text generation step.

In [ ]:
pip install openai

In [ ]:
# Hide your api-key using getpass
from openai import OpenAI
import json
from getpass import getpass

api_key = getpass('Enter your API key: ')
client = OpenAI(api_key=api_key)

Enter your API key: ··········


In [ ]:
# Here is the function for text generation with developer instructions that led to the best results.
def chatgpt(examples):
    completion = client.chat.completions.create(
        model="chatgpt-4o-latest", # the best model for Russian according to the Hugging Face Arena
        messages=[
            {"role": "developer", "content": "You are a Russian journalist. I will provide you with some examples of news articles in a moment. You will be asked to create a news article that matches the topic and writing style. The length of the article should be at least 400 words."},
            {"role": "assistant", "content": "Sure, please provide the example news articles, and I’ll create a similar one for you."},
            {"role": "user", "content": construct_example_prompt(examples)}
        ],
    )

    y = json.loads(str(completion.model_dump_json()), strict=False)
    response = y["choices"][0]["message"]["content"]

    return response

In [ ]:
# The text quality seems satisfactory.
print(chatgpt(example_text(df.texts.values, num=2)))

Учёные обнаружили новый штамм гриппа в России

Российские учёные зафиксировали новый штамм вируса гриппа, который вызывает атипичные симптомы и распространяется с высокой скоростью. По заявлению Роспотребнадзора, за последние две недели в стране зарегистрировано более 250 случаев заражения новым штаммом, наиболее острые случаи отмечены в Центральном и Северо-Западном федеральных округах.

"Вирус был идентифицирован как разновидность гриппа типа A, но он отличается от ранее фиксировавшихся штаммов. На данный момент проводятся дополнительные исследования для уточнения его особенностей и источника происхождения", — заявил представитель ведомства на пресс-конференции.

Симптомы инфекции включают резкое повышение температуры, сильные боли в суставах и мышцах, а также отечность горла, что необычно для стандартных форм гриппа. Медики предупреждают, что заболевание может иметь более тяжёлое течение у пожилых людей и лиц с хроническими заболеваниями.

По словам представителей медицинского сообщ

In [ ]:
text_output = chatgpt(example_text(df.texts.values, num=2))

In [ ]:
print(text_output)

Полиция задержала мужчину, подозреваемого в нападении на туристов в Казани  

В Казани правоохранители задержали мужчину, подозреваемого в нападении на иностранных туристов на территории исторического центра города. Об этом РИА Новости сообщил источник в полиции.  

Инцидент произошел днем 12 октября вблизи Казанского Кремля. По словам очевидцев, неизвестный нанес несколько ударов двум мужчинам из числа туристов, которые делали фотографии архитектурного комплекса. Один из пострадавших получил травму головы, второй — порезы руки.  

«Мужчина вел себя агрессивно, выкрикивал нецензурные выражения и ударил одного из иностранных гостей бутылкой по голове», — рассказал очевидец.  

Сразу после происшествия на место прибыли сотрудники полиции и бригада скорой помощи. Пострадавшие были доставлены в ближайшую больницу, где им оказали медицинскую помощь. Оба пострадавших находятся в стабильном состоянии, их жизни ничего не угрожает.  

Подозреваемый, предположительно, является местным жителем, в

In [ ]:
# Re-using the same function for calculating the word counts in the generated texts.
def word_count(text):
    '''
    Tokenizes the text into words, excludes punctuation but retains the numbers, and counts word tokens.
    Returns the word tokens.
    '''

    tokens = word_tokenize(text)
    word_tokens = [word for word in tokens if word.isalnum()]

    return len(word_tokens)

In [ ]:
import nltk
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [ ]:
# This length is the result of multiple experiments.
# Still the text is shorter than our target goal but already close enough to the mean word count in human texts.
word_count(text_output)

274

We will save the model's output into a pandas dataframe for further use, following the structure of our other datasets.

This cell takes about an hour to run!

In [ ]:
text = []  # output of the AI
author = []  # model
ai = []  # 0 for human class, 1 for AI class

model = "chatgpt-4o-latest"

for i in tqdm(range(0, 500), desc="Generating AI Texts"):
    text.append(chatgpt(example_text(df.texts.values, 2)))
    author.append(model)
    ai.append(1)

dfAI = pd.DataFrame({'texts': text, 'source': author, 'class': ai})

Generating AI Texts: 100%|██████████| 500/500 [1:02:45<00:00,  7.53s/it]


In [ ]:
# Verifying the number of total output texts
len(dfAI)

500

In [ ]:
dfAI.head(10)

,texts,source,class
0,В Москве началась реализация новой экологическ...,chatgpt-4o-latest,1
1,Пошлина на экспорт зерновых из России снижена ...,chatgpt-4o-latest,1
2,Президент России наградил волонтера за помощь ...,chatgpt-4o-latest,1
3,В Москве разгорелись споры из-за заявления Соб...,chatgpt-4o-latest,1
4,Путин поддержал инициативу о создании нового э...,chatgpt-4o-latest,1
5,Потоп в Дагестане: ущерб достигает миллиардов ...,chatgpt-4o-latest,1
6,Певец Стивен Тайлер оказался в центре скандала...,chatgpt-4o-latest,1
7,Производители искусственного мяса расширяют ры...,chatgpt-4o-latest,1
8,Ученые предсказали последствия таяния ледников...,chatgpt-4o-latest,1
9,В Санкт-Петербурге эвакуировали 200 человек из...,chatgpt-4o-latest,1


In [ ]:
print(dfAI['texts'][4])

Путин поддержал инициативу о создании нового экологического фонда в России

Президент России Владимир Путин одобрил предложение о создании нового экологического фонда, направленного на поддержку проектов по защите и восстановлению окружающей среды. Об этом сообщила пресс-служба Кремля по итогам заседания Совета по экологии и климату, прошедшего сегодня в Москве.

Как отметил президент в ходе заседания, защита природы должна оставаться одним из приоритетов государственной политики. Он подчеркнул необходимость активного участия бизнеса, общественных организаций и научного сообщества в реализации экологических инициатив. Новый фонд, по его словам, станет платформой для объединения усилий государственных и частных структур.

«Россия обладает уникальной природой, и наша обязанность — сохранить её для будущих поколений. Новый экологический фонд позволит не только аккумулировать финансовые ресурсы, но и направлять их на важнейшие проекты, касающиеся воспроизводства лесов, очистки водоёмов и с

In [ ]:
tqdm.pandas()

dfAI['word_counts'] = dfAI['texts'].progress_apply(word_count)

100%|██████████| 500/500 [00:02<00:00, 200.94it/s]


In [ ]:
# On average the texts are a shorter than the inteded goal.
dfAI['word_counts'].describe()

,word_counts
count,500.00000
mean,279.03200
std,42.32325
min,6.00000
25%,272.00000
50%,285.00000
75%,297.00000
max,343.00000


In [ ]:
dfAI['genre'] = 'news'
dfAI = dfAI[['texts', 'source', 'word_counts', 'genre', 'class']]

In [ ]:
dfAI.sample(2)

,texts,source,word_counts,genre,class
190,Мэрия Москвы вводит новые правила для электрос...,chatgpt-4o-latest,284,news,1
453,Петербургский «Зенит» стал чемпионом России по...,chatgpt-4o-latest,261,news,1


In [ ]:
# Save the output to file.
dfAI.to_csv('data/original/ai/ai_news.csv', index=False, encoding='utf-8')